# Importations

In [14]:
import pandas as pd

# Step 1: Load Data into Pandas

In [22]:
# Load dataset files into DataFrames
df_active = pd.read_csv("Flipside Dataset/flipside_days_active.csv")
df_labels = pd.read_csv("Flipside Dataset/flipside_dim_labels.csv", sep="\t")
df_new_programs = pd.read_csv("Flipside Dataset/Solana Weekly New Program.csv")
df_new_users = pd.read_csv("Flipside Dataset/Solana Weekly New Users.csv")
df_unique_programs = pd.read_csv("Flipside Dataset/Solana Weekly Unique Programs.csv")
df_unique_users = pd.read_csv("Flipside Dataset/Weekly Unique Users.csv")
df_days_since_last_use = pd.read_csv("Flipside Dataset/weekly_days_since_last_use.csv")
df_users_last_use = pd.read_csv("Flipside Dataset/Weekly_users_last_use.csv")


# Step 2: Merge Tables Based on Common Keys

**I. Merge User Activity with Labels**

In [26]:
# Keep only the first 6 valid columns
df_labels = df_labels.iloc[:, :6]  
# Rename Columns 
df_labels.columns = ["BLOCKCHAIN", "CREATOR", "ADDRESS", "LABEL_TYPE", "LABEL_SUBTYPE", "ADDRESS_NAME"]

In [36]:
df_merged = df_active.merge(df_labels, left_on="SIGNER", right_on="ADDRESS", how="left")

In [39]:
print(df_merged).head()

                                              SIGNER            FIRST_TX_DATE  \
0       9P65j1MLdkve3iShPXF3H6F32qGhKYmVLyza5J4ZvkyU  2025-03-11 00:00:00.000   
1       6VvDGnnAeEeK3yd4jAdH5X7ZdRVTsrbrg4beDZvQemmG  2025-03-11 00:00:00.000   
2        gGf97pe7apxKNmQyiBctckiLeuPTBrbh4HridK6bNBz  2025-03-11 00:00:00.000   
3       7VEETqsSS9QdhZZg2fZcpivzLYg8zbQrKEzpFoN9zpav  2025-03-11 00:00:00.000   
4       ESJkFSHeeHE4rBHKxVAKA4QxjL93KPH7MbWrwkyDQgQx  2025-03-11 00:00:00.000   
...                                              ...                      ...   
99996   5gog9z4sMbYQUp9dgwxf77NsxzZn8nZhLoPhw5SbyEtM  2025-03-11 00:00:00.000   
99997   7EHyYjXtqfepRdstb8HZghz2yRHVCE5v7jTFbcqwConi  2025-03-11 00:00:00.000   
99998   3692PsUoikuixzWTHgg7CW43ANLsJH6FpuRU9zwkgaMX  2025-03-11 00:00:00.000   
99999   Ds9ce5uN4tGEMdMPw7jVFzrJLK71anR2kCMM79p8NuU3  2024-10-31 00:00:00.000   
100000                                             ﻿                      NaN   

                           

AttributeError: 'NoneType' object has no attribute 'head'

**II. Add Days Since Last Use**

In [38]:
print(df_days_since_last_use).head()


    CREATION_DATE  Days since last use  Days since creation
0      2020-10-07           633.616567           750.556628
1      2020-10-14           611.422904           743.556628
2      2020-10-21           528.263984           736.556628
3      2020-10-28           492.900152           729.556628
4      2020-11-04           696.768517           722.556628
..            ...                  ...                  ...
103    2022-09-28            23.075857            29.556628
104    2022-10-05            18.225040            22.556628
105    2022-10-12            12.097492            15.556628
106    2022-10-19             6.609546             8.556628
107    2022-10-26             0.994051             1.556628

[108 rows x 3 columns]


AttributeError: 'NoneType' object has no attribute 'head'

In [32]:
df_merged = df_merged.merge(df_days_since_last_use, left_on="SIGNER", right_on="ADDRESS", how="left")


KeyError: 'ADDRESS'